In [24]:
function accumarray(subs, val, fun=sum, fillval=0.0; sz=maximum(subs,1), issparse=false)
    counts = Dict{Array{Int64,2},Vector{Float64}}()
  
   for i in 1:size(subs,1)
        counts[subs[i,:]]=push!(get(counts,subs[i,:],Vector{Float64}()) , val[i])
    end
    A = fill(fillval,sz...)

   
   for j = keys(counts)
        A[j...] = fun(counts[j])
   end
   issparse ? sparse(A) : A
end

accumarray (generic function with 3 methods)

In [59]:
val = int8(10:15);
subs = [1 1 1; 1 1 1; 1 1 2; 1 1 2; 2 3 1; 2 3 2]
accumarray(subs,val)

2x3x2 Array{Float64,3}:
[:, :, 1] =
 21.0  0.0   0.0
  0.0  0.0  14.0

[:, :, 2] =
 25.0  0.0   0.0
  0.0  0.0  15.0

In [44]:
v=[100.1;101.2]
var(v)

0.6050000000000093

In [49]:
val = [100.1 101.2 103.4 102.8 100.9 101.5];
subs = [1 1; 1 1; 2 2; 3 2; 2 2; 3 2]
#accumarray(subs, val, v->norm(v-mean(v))^2)
accumarray(subs, val, fft)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::Array{Complex{Float64},1})
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[49], in expression starting on line 4

In [52]:
v=[1,1]
fft(v)

2-element Array{Complex{Float64},1}:
 2.0+0.0im
 0.0+0.0im

In [53]:
subs[1,:]

1x2 Array{Int64,2}:
 1  1

In [62]:
A=sprandn(3,4,.5); x=randn(4)

4-element Array{Float64,1}:
 -1.12307 
 -0.550569
  1.70355 
  0.343585

In [63]:
A*x

3-element Array{Float64,1}:
 -2.00403
  1.64066
  0.92041

In [94]:
z = [  ((i,j),A[i,j]) for i=1:3, j=1:4 ][:]

f= y -> (y[1][1], y[2] * x[y[1][2]])

pairs = map(f, z)
     
subs = [ Int64(pairs[i][1]) for i=1:12]
vals = [ Float64(pairs[i][2]) for i=1:12]
accumarray(subs,vals)

3-element Array{Float64,1}:
 -2.00403
  1.64066
  0.92041

In [95]:
A*x

3-element Array{Float64,1}:
 -2.00403
  1.64066
  0.92041

In [1]:
using SymPy

[Plots.jl] Default backend: pyplot

In [2]:
function gather{K,V}(data::Vector{Tuple{K,V}})
    D= Dict{K,Vector{V}}()
    for d in data
        D[d[1]] = haskey(D,d[1]) ? push!(D[d[1]],d[2]) : [d[2]] 
    end
    D
end

function reducer(D,f) 
    vcat( ([(d[1]==0  || length(d[2])==1 ) ? d[2] :   f(d[2]) for d in D]) ...)
end

⊕(x,y) = x[1]<y[1] ? [(x[1],x[2]),(y[1],x[2]+y[2])] : [(y[1],y[2]),(x[1],y[2]+x[2])]
⊕(x) = x[1] ⊕ x[2]

⊕ (generic function with 2 methods)

In [25]:
n=8
data=[(i,Sym(string('a'+i-1))) for i=1:n]
@show data
data = map( kv -> prefixmap(kv,1), data)  |> gather   |>  d->reducer(d,⊕)
data = map( kv -> prefixmap(kv,2), data)  |> gather   |>  d->reducer(d,⊕)
data = map( kv -> prefixmap(kv,4), data)  |> gather   |>  d->reducer(d,⊕)
data = map( kv -> prefixmap(kv,2,2), data)  |> gather   |>  d->reducer(d,⊕)
data = map( kv -> prefixmap(kv,1,1), data)  |> gather   |>  d->reducer(d,⊕)

data = Tuple{Any,Any}[(1,a),(2,b),(3,c),(4,d),(5,e),(6,f),(7,g),(8,h)]

8-element Array{Tuple{Int64,SymPy.Sym},1}:
 (1,a)                            
 (8,a + b + c + d + e + f + g + h)
 (4,a + b + c + d)                
 (5,a + b + c + d + e)            
 (6,a + b + c + d + e + f)        
 (7,a + b + c + d + e + f + g)    
 (2,a + b)                        
 (3,a + b + c)                    

In [7]:
n=4
data=[(i,Sym(string('a'+i-1))) for i=1:n]
prefixmap(kv,n,offset=0) = (kv[1] % n != 0 ? 0 : Int(ceil((kv[1]-offset)/2n)), kv )

@show data 
for i=0:floor(log2(n))
    prefixmap(kv) = prefixmap(kv,2^i)
    data = map(prefixmap,data)
    @show data
    data = gather(data)
    
    @show data
    data = reducer(data,⊕)
    @show data
end


for i=floor(log2(n)*2/3):-1:0
    prefixmap(kv) = prefixmap(kv,2^i,2^i)
    data = map(prefixmap,data)
    data = gather(data)
    data = reducer(data,⊕)
end
    
data

data = Tuple{Any,Any}[(1,a),(2,b),(3,c),(4,d)]

4-element Array{Tuple{Int64,SymPy.Sym},1}:
 (1,a)            
 (4,a + b + c + d)
 (2,a + b)        
 (3,a + b + c)    


data = [(1,(1,a)),(1,(2,b)),(2,(3,c)),(2,(4,d))]
data = Dict(2=>[(3,c),(4,d)],1=>[(1,a),(2,b)])
data = [(3,c),(4,c + d),(1,a),(2,a + b)]
data = [(0,(3,c)),(1,(4,c + d)),(0,(1,a)),(1,(2,a + b))]
data = Dict(0=>[(3,c),(1,a)],1=>[(4,c + d),(2,a + b)])
data = [(3,c),(1,a),(2,a + b),(4,a + b + c + d)]
data = [(0,(3,c)),(0,(1,a)),(0,(2,a + b)),(1,(4,a + b + c + d))]
data = Dict(0=>[(3,c),(1,a),(2,a + b)],1=>[(4,a + b + c + d)])
data = [(3,c),(1,a),(2,a + b),(4,a + b + c + d)]


In [4]:
3 % 2

1

In [5]:
reducer(data2,⊕)

LoadError: LoadError: UndefVarError: data2 not defined
while loading In[5], in expression starting on line 1

In [6]:
data[2][2] ⊕ data[1][2]

LoadError: LoadError: BoundsError
while loading In[6], in expression starting on line 1

In [7]:
data=[ ("a","dog"),("a","hello")]

2-element Array{Tuple{ASCIIString,ASCIIString},1}:
 ("a","dog")  
 ("a","hello")

In [8]:
gather(data)["a"]

2-element Array{ASCIIString,1}:
 "dog"  
 "hello"

In [9]:
Int('a')

97

In [10]:
string(char(97))

"a"

In [11]:
'a'-1

'`'

 in depwarn at /Applications/Julia-0.4.0-rc1.app/Contents/Resources/julia/lib/julia/sys.dylib
 in char at deprecated.jl:50
 in include_string at loading.jl:233
 in execute_request_0x535c5df2 at /Users/alanedelman/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/alanedelman/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:448
while loading In[10], in expression starting on line 1


In [66]:
floor(log2(8))

3.0

In [8]:
A=rand(3,3)

3x3 Array{Float64,2}:
 0.100397  0.99484   0.31888 
 0.328299  0.604748  0.621747
 0.694615  0.170051  0.212911

In [10]:
@which( lufact(A))

lufact{T<:AbstractFloat}(A::Union{AbstractArray{Complex{T<:AbstractFloat},2},AbstractArray{T<:AbstractFloat,2}}) at linalg/lu.jl:70

In [11]:
@which( eigvals(A))

eigvals{T}(A::Union{DenseArray{T,2},SubArray{T,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/eigen.jl:84

In [13]:
? LAPACK.geevx!

In [14]:
methods(LAPACK.geevx!)

# 4 methods for generic function "geevx!":
geevx!(balanc::Char, jobvl::Char, jobvr::Char, sense::Char, A::Union{DenseArray{Float64,2},SubArray{Float64,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/lapack.jl:1279
geevx!(balanc::Char, jobvl::Char, jobvr::Char, sense::Char, A::Union{DenseArray{Float32,2},SubArray{Float32,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/lapack.jl:1279
geevx!(balanc::Char, jobvl::Char, jobvr::Char, sense::Char, A::Union{DenseArray{Complex{Float64},2},SubArray{Complex{Float64},2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/lapack.jl:1426
geevx!(balanc::Char, jobvl::Char, jobvr::Char, sense::Char, A::Union{DenseArray{Complex{Float32},2},SubArray{Complex{Float32},2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/lapack.jl:1426

In [19]:
L=Sym("L"); A=Sym("A"); P=Sym("P"); C=Sym("C"); K=Sym("K")
L=1
A=1
P=1
C=1
K=1

1

In [20]:

M= [L A P A C K 
 L -A P -A C -K
 L A P A -C -K
L -A P -A -C K
L A -P -A C K
L -A -P A C -K]

6x6 Array{Int64,2}:
 1   1   1   1   1   1
 1  -1   1  -1   1  -1
 1   1   1   1  -1  -1
 1  -1   1  -1  -1   1
 1   1  -1  -1   1   1
 1  -1  -1   1   1  -1

In [22]:
4*inv(M)

6x6 Array{Float64,2}:
 0.0   0.0   1.0   1.0   1.0   1.0
 0.0   0.0   1.0  -1.0   1.0  -1.0
 1.0   1.0   0.0  -0.0  -1.0  -1.0
 1.0  -1.0   0.0  -0.0  -1.0   1.0
 1.0   1.0  -1.0  -1.0   0.0   0.0
 1.0  -1.0  -1.0   1.0   0.0   0.0

In [28]:
Pkg.clone("Elemental")

INFO: Cloning Elemental from git://github.com/JuliaParallel/Elemental.jl.git
INFO: Computing changes...
INFO: Downgrading DistributedArrays: v0.1.6 => v0.1.5


In [29]:
Pkg.build("Elemental")

INFO: Building Elemental
Cloning into '/Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental'...


-- The C compiler identification is AppleClang 7.0.0.7000072
-- The CXX compiler identification is AppleClang 7.0.0.7000072
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done


CMake Warning at CMakeLists.txt:238 (message):
  Build mode not specified, defaulting to Release build.




-- Could NOT find PythonInterp (missing:  PYTHON_EXECUTABLE) 
-- Appending /Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental/include for Elemental's source includes
-- Appending /Users/alanedelman/.julia/v0.4/Elemental/deps/builds/include for Elemental's binary includes
-- The Fortran compiler identification is GNU 5.2.0
-- Checking whether Fortran compiler has -isysroot
-- Checking whether Fortran compiler has -isysroot - yes
-- Checking whether Fortran compiler supports OSX deployment target flag
-- Checking whether Fortran compiler supports OSX deployment target flag - yes
-- Check for working Fortran compiler: /usr/local/bin/gfortran
-- Check for working Fortran compiler: /usr/local/bin/gfortran  -- works
-- Detecting Fortran compiler ABI info
-- Detecting Fortran compiler ABI info - done
-- Checking whether /usr/local/bin/gfortran supports Fortran 90
-- Checking whether /usr/local/bin/gfortran supports Fortran 90 -- yes
-- Detecting Fortran/C Interface
-- Detecting Fortr

CMake Warning:
  Manually-specified variables were not used by the project:

    PYTHON_SITE_PACKAGES



...

[ 12%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/lapack/odssq.c.o
[ 12%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/lapack/odstmr.c.o
[ 12%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/lapack/oerbla.c.o
[ 12%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/lapack/olsame.c.o
[ 12%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/plarre.c.o
[ 13%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/plarrv.c.o
[ 13%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/pmrrr.c.o
[ 13%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/process_c_task.c.o
[ 13%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/process_r_task.c.o


4 warnings generated.
10 warnings generated.
/Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental/external/pmrrr/src/lapack/odstmr.c:379:34: warning: '&&' within '||' [-Wlogical-op-parentheses]
    } else if (*ldz < 1 || wantz && *ldz < *n) {
                        ~~ ~~~~~~^~~~~~~~~~~~
/Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental/external/pmrrr/src/lapack/odstmr.c:379:34: note: place parentheses around the '&&' expression to silence this warning
    } else if (*ldz < 1 || wantz && *ldz < *n) {
                                 ^
                           (                 )
/Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental/external/pmrrr/src/lapack/odstmr.c:460:34: warning: '&&' within '||' [-Wlogical-op-parentheses]
        if (alleig || valeig && r2 > wl && r2 <= wu || indeig && iil == 1) {
                   ~~ ~~~~~~~~~~~~~~~~~~^~~~~~~~~~~
/Users/alanedelman/.julia/v0.4/Elemental/deps/src/Elemental/external/pmrrr/src/lapack/odstmr.c:460:34: note: p

[ 13%] Building C object external/pmrrr/CMakeFiles/pmrrr.dir/src/process_s_task.c.o


5 warnings generated.


[ 13%] Linking C shared library libpmrrr.dylib
[ 13%] Built target pmrrr
Scanning dependencies of target El
[ 13%] Building C object CMakeFiles/El.dir/src/core/global.c.o
[ 13%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AdjointAxpyContract.cpp.o
[ 13%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/Adjoint.cpp.o
[ 13%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AdjointAxpy.cpp.o
[ 13%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AdjointContract.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/Axpy/util.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AllReduce.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/Axpy.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AxpyContract.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/Broadcast.cpp.o
[ 14%] Building CXX object CMakeFiles/El.dir/src/blas_like/level1/AxpyTrapezo

In [30]:
1+1

2

In [31]:
using Elemental